In [ ]:
# 加载环境变量，如果你用的是OpenAI key就用这个方案！去掉注释即可！后面定义本地model的时候，把它注释
# import dotenv
# %load_ext dotenv
# %dotenv

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.core import SimpleDirectoryReader

# 加载PDF数据，这里雄哥以卖油翁为例，可以改为自己的任意数据放在datasets文件夹即可，或改为指定的绝对路径
documents = SimpleDirectoryReader(input_files=["./datasets/maiyouweng.pdf"]).load_data()

In [3]:
from llama_index.core.node_parser import SentenceSplitter

# 把文本分块，chunk_size为1024，每一块的大小
splitter = SentenceSplitter(chunk_size=1024)
# 创建文档的节点
nodes = splitter.get_nodes_from_documents(documents)

In [4]:
# 这里可以打印每一个块的信息！
node_metadata = nodes[1].get_content(metadata_mode=True)
print(node_metadata)

page_label: 1
file_name: maiyouweng.pdf
file_path: datasets\maiyouweng.pdf
file_type: application/pdf
file_size: 429687
creation_date: 2024-06-13
last_modified_date: 2024-04-29

尔，同“耳”，相当于“罢了”。
12.忿（fèn）然：气愤的样子。然，作形容词或者副词的词尾，相当于“的”或“地”。
13.安：怎么。轻吾射：看轻我射箭（的本领）。轻，作动词用。
14.以我酌（zhuó）油知之：凭我倒油（的经验）知道这个（道理）。以，凭、靠。
酌，斟酒，这里指倒油。之，指射箭也是凭手熟的道理。
15.覆：盖。
16.徐：慢慢地。
17.杓：同“勺”。
18.沥之：注入葫芦。沥，注。之，指葫芦。
19.惟：意为只，不过。
20.遣之：让他走。遣，打发。


In [5]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

C:\Users\ouhui\.conda\envs\xuehai3\lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [6]:
# 加载本地的qwen2-7b，你本地的模型放在哪就直接改路径
llm = HuggingFaceLLM(
    tokenizer_name="G:\\models\\Qwen2-7B-Instruct",
    model_name="G:\\models\\Qwen2-7B-Instruct",
    device_map="auto",
    tokenizer_kwargs={"trust_remote_code": True},
    model_kwargs={"trust_remote_code": True},
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# 调用本地bce-embedding-base_v1作为embedding模型

embed_args = {
    'model_name': 'hkunlp/instructor-base', 
    'max_length': 512, 
    'embed_batch_size': 32, 
    'device': 'cuda'
    }
embed_model = HuggingFaceEmbedding(**embed_args)

C:\Users\ouhui\.conda\envs\xuehai3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
from llama_index.core import Settings
# 把加载的模型添加到环境设置中，定义全局模型

Settings.llm = llm
Settings.embed_model = embed_model

In [9]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# 创建数据摘要索引
summary_index = SummaryIndex(nodes)
# 创建矢量存储索引
vector_index = VectorStoreIndex(nodes)

In [55]:
def add(x: int, y: int) -> int:
    """将两个数相加"""
    return x + y

# substraction function
def sub(x: int, y: int) -> int:
    """两个数字相减"""
    return x - y

# multiplication function
def mul(x: int, y: int) -> int:
    """两个数字相乘"""
    return x * y


# get user information
def get_user_info(name: str) -> str:
    """Get user information."""
    data = {
        "雄哥": {
            "age": 18,
            "location": "广东"
        },
        "小胖": {
            "age": 60,
            "location": "广东"
        }
    }
    return f'名字 {name}, 年龄 {data[name]["age"]} 来自 {data[name]["location"]}'

In [56]:
from llama_index.core.tools import FunctionTool

addition_tool = FunctionTool.from_defaults(fn=add)
get_user_info_tool = FunctionTool.from_defaults(fn=get_user_info)
multiplication_tool = FunctionTool.from_defaults(fn=mul)
substraction_tool = FunctionTool.from_defaults(fn=sub)

tools = [addition_tool, get_user_info_tool, multiplication_tool, substraction_tool]

In [42]:
# 这里试试上面定义的函数，大模型能否正常使用
response = llm.predict_and_call(
    tools, 
    "5乘以5等于多少？", 
    verbose=True
)
print(str(response))

Thought: 用户需要计算5乘以5的结果。我有一个mul工具可以用来完成这个任务。
Action: mul
Action Input: {'x': 5, 'y': 5}
Observation: 25
25


In [57]:
# 再试一下，确定能正常使用
response = llm.predict_and_call(
    tools, 
    "雄哥多少岁？", 
    verbose=True
)
print(str(response))

Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: get_user_info
Action Input: {'name': '雄哥'}
Observation: 名字 雄哥, 年龄 18 来自 广东
名字 雄哥, 年龄 18 来自 广东


In [16]:
from llama_index.core.vector_stores import MetadataFilters

# Create vector search query engine
query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "康肃公陈尧咨善于射箭，曾经，他在家里场地射箭，然后发生了什么事？", 
)
print(str(response))

陈尧咨公陈尧咨擅长射箭，他的技艺无人能及，因此他颇为自满。一次，他在自家的射箭场练习射箭时，有一个卖油的老翁放下担子，站在一边注视着他。老翁看了陈尧咨射箭，发现他射箭十箭中了八九箭，只是微微地点点头。陈尧咨便询问老翁是否也懂射箭，并夸耀自己的射箭技巧高超。然而，老翁只回答说：“这没什么特别的，只是手法熟练罢了。”陈尧咨听后大怒，认为老翁轻视了他的射箭技术。老翁则解释说，他凭倒油的经验就能理解这个道理，并当场演示了如何用勺子倒油穿过一枚覆盖在地上的铜钱的中心，而铜钱却不被沾湿。最后，老翁说，他之所以能做到这一点，也是因为他的手很熟练。陈尧咨听后笑了，明白了老翁的意思，便笑着让老翁离开了。这段故事后来被称作“卖油翁”的典故，寓意着技艺高超的人往往是因为他们反复练习和熟悉操作过程的结果。


In [15]:
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text())
    print("=============Text=============")

{'page_label': '2', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
我射箭的技术不也很精妙吗？”卖油的老翁说：
“（这）没有别的（奥妙），只是手法熟练罢了。”
5、康肃忿然曰：“尔安敢轻吾射！”
翁曰：“以我酌油知之。”
句译：陈尧咨（听后）气愤地说：“你怎么敢轻视我射箭的本领！”老翁说：“凭我倒油的经
验知道这个道理。”
6、乃取一葫芦置于地，以钱覆其口，徐以杓酌油沥之，自钱孔入，而钱不湿。
句译：于是（卖油翁）拿一个葫芦放在地上，用一枚铜钱盖住葫芦口，用勺子慢慢地将油向
下灌注（到葫芦里）。油从钱孔中注入葫芦，但是铜钱却没有沾湿。
7、因曰：“我亦无他，惟手熟尔。”康肃笑而遣之。
句译：于是他说：“我也没有别的奥妙，只是手法熟练罢了。”康肃公笑着打发他走了。
六、字词释义
=============Text=============
{'page_label': '2', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
四、白话译文
康肃公陈尧咨善于射箭，世上没有第二个人能跟他相媲美，他也就凭着这种本领而自夸。
曾经（有一次），（他）在家里（射箭的）场地射箭，有个卖油的老翁放下担子，站在那里
斜着眼睛看着他，很久都没有离开。卖油的老头看他射十箭中了八九箭，但只是微微点点头。
陈尧咨问卖油翁：“你也懂得射箭吗？我的箭法不是很高明吗？”卖油的老翁说：“没有
别的（奥

In [17]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_search_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """使用以下参数在索引中进行向量搜索：

    query (str): 这是你想要在索引中嵌入和搜索的文本字符串
    page_numbers (List[str]): 这个参数允许你将搜索限制到特定的页面。如果留空，搜索将包含索引中的所有页面。如果指定了页码，搜索将只包括那些页面
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

In [36]:
vector_query_tool = FunctionTool.from_defaults(
    name="vector_search_tool",
    fn=vector_search_query
)

In [22]:
response = llm.predict_and_call(
    [vector_query_tool], 
    "第2页讲了什么？", 
    verbose=True
)

Thought: 用户询问的是关于某本书或文档的特定页面内容。我需要使用vector_search_tool工具来查找相关信息。
Action: vector_search_tool
Action Input: {'query': '第二页', 'page_numbers': ['2']}
Observation: 该页面内容涉及《卖油翁》的翻译和重点句子解释。其中，陈康肃公陈尧咨擅长射箭，并因此自我夸耀。他曾在自家的园子里射箭时被一个卖油的老翁观察。老翁对陈尧咨的箭术有所评价，但并不表示赞同或否定。陈尧咨质疑老翁是否懂射箭，并对老翁的评价表示不满，认为自己的箭术高超。老翁则解释自己只是通过倒油的经验了解到了技巧的熟练性，并演示了如何将油从铜钱的孔中倒入葫芦而不使铜钱沾湿，以此证明自己的观点。最后，陈尧咨认识到老翁的技艺并表示理解，两人以微笑结束这次交流。字词释义部分未给出，需要具体分析每个关键词的含义，包括但不限于“善射”、“自矜”、“尝”、“释担而立”、“睨”、“但微颔之”等。


In [23]:
# 确认系统在哪里检索的数据，返回数据源
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text())
    print("=============Text=============")

{'page_label': '2', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
四、白话译文
康肃公陈尧咨善于射箭，世上没有第二个人能跟他相媲美，他也就凭着这种本领而自夸。
曾经（有一次），（他）在家里（射箭的）场地射箭，有个卖油的老翁放下担子，站在那里
斜着眼睛看着他，很久都没有离开。卖油的老头看他射十箭中了八九箭，但只是微微点点头。
陈尧咨问卖油翁：“你也懂得射箭吗？我的箭法不是很高明吗？”卖油的老翁说：“没有
别的（奥妙），不过是手法熟练罢了。”陈尧咨（听后）气愤地说：“你怎么敢轻视我射箭（的
本领）！”老翁说：“凭我倒油（的经验）就可以懂得这个道理。”于是拿出一个葫芦放在地
上，把一枚铜钱盖在葫芦口上，慢慢地用油杓舀油注入葫芦里，油从钱孔注入而钱却没有湿。
于是说：“我也没有别的（奥妙），只不过是手熟练罢了。”陈尧咨笑着将他送走了。
五、重点句子
1、陈康肃公善射，当世无双，公亦以此自矜。
句译：康肃公陈尧咨擅长射箭，当时没有人能与他相比，他（陈尧咨）也因此自我夸耀。
2、尝射于家圃，有卖油翁释担而立，睨之久而不去。
句译：他曾经（有一次）在家里的园子里射箭，有一个卖油老头放下担子站在那儿，斜着眼
看他射箭，很久没有离开。
3、见其发矢十中八九，但微颔之。
句译：（卖油翁）看见他射出的箭十支有八九支射中了目标，只是微微点头。
4、康肃问曰：“汝亦知射乎？吾射不亦精乎？”
翁曰：“无他，但手熟尔。”
句译：陈尧咨问卖油翁：“你也懂得射箭吗？我射箭的技术不也很精妙吗？”卖油的老翁说：
“（这）没有别的（奥妙），只是手法熟练罢了。”
5、康肃忿然曰：“尔安敢轻吾射！”
翁曰：“以我酌油知之。”
句译：陈尧咨（听后）气愤地说：“你怎么敢轻视我射箭的本领！”老翁说：“凭我倒油的经
验知道这个道理。
=============Text=============
{'pag

In [68]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "关于卖油翁的摘要生成"
    ),
)

In [71]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "第一页提到了什么内容?",
    verbose=True
)

Thought: The user is asking about the content on the first page. I need to use the `vector_search_tool` to search for the content on the first page.
Action: vector_search_tool
Action Input: {'query': '第一页', 'page_numbers': ['1']}
Observation: 《卖油翁》是人教版社初中语文七年级下册第三单元的第12课。文章通过卖油翁与陈尧咨的对话及卖油翁酌油的事例，说明了熟能生巧的道理。文章包含详细的故事背景、人物介绍、原文文本以及词语注释。


In [72]:
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text()[:10])
    print("=============Text=============")

{'page_label': '1', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
《卖油翁》
一、教材
=============Text=============
{'page_label': '1', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
尔，同“耳”，相当于
=============Text=============


In [73]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "给我这篇卖油翁课堂的摘要！",
    verbose=True
)

Thought: 我需要使用一个工具来生成关于“卖油翁”课堂的摘要。
Action: summary_tool
Action Input: {'input': '卖油翁课堂'}
Observation: 课堂上教授的《卖油翁》内容涉及以下几个方面：

1. **文章背景**：《卖油翁》是欧阳修为宋英宗治平四年（1067年）所作，其写作背景是因为欧阳修遭人中伤，自请外任时创作了这篇短文。

2. **主要人物**：文章主要讲述了陈尧咨与卖油翁的故事。陈尧咨是宋代的官员，以其射箭技艺自豪；而卖油翁则以其倒油技巧闻名，通过与陈尧咨的互动，展现了“熟能生巧”的道理。

3. **故事主题**：通过陈尧咨的射箭技巧与卖油翁的倒油技巧的对比，强调了技能通过不断练习可以达到高超水平的观念。文章旨在教导学生正确对待自己的长处和他人的长处，以及谦虚的重要性。

4. **课堂讨论**：课堂上可能还涉及对人物性格的分析、对待成功和才能的态度探讨，以及引用相关名言，如“三人行，必有我师焉”和“谦虚使人进步，骄傲使人落后”，以深化学生对文章主题的理解。

通过这些内容的学习，学生们不仅


In [74]:
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text()[:10])
    print("=============Text=============")

{'page_label': '1', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
《卖油翁》
一、教材
=============Text=============
{'page_label': '1', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
尔，同“耳”，相当于
=============Text=============
{'page_label': '2', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 'creation_date': '2024-06-13', 'last_modified_date': '2024-04-29'}
=============Text=============
四、白话译文
康肃公
=============Text=============
{'page_label': '2', 'file_name': 'maiyouweng.pdf', 'file_path': 'datasets\\maiyouweng.pdf', 'file_type': 'application/pdf', 'file_size': 429687, 